In [1]:
import pandas as pd
import numpy as np
import joblib
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [2]:
# STEP 1: Load Soil Dataset
soil_df = pd.read_csv("Crop_recommendation.csv", usecols=["N", "P", "K", "ph", "label"])

In [3]:
soil_df.head()

,N,P,K,ph,label
0,90,42,43,6.502985,rice
1,85,58,41,7.038096,rice
2,60,55,44,7.840207,rice
3,74,35,40,6.980401,rice
4,78,42,42,7.628473,rice


In [9]:
soil_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   N       2200 non-null   int64  
 1   P       2200 non-null   int64  
 2   K       2200 non-null   int64  
 3   ph      2200 non-null   float64
 4   label   2200 non-null   object 
dtypes: float64(1), int64(3), object(1)
memory usage: 86.1+ KB


In [10]:
soil_df.describe()

,N,P,K,ph
count,2200.000000,2200.000000,2200.000000,2200.000000
mean,50.551818,53.362727,48.149091,6.469480
std,36.917334,32.985883,50.647931,0.773938
min,0.000000,5.000000,5.000000,3.504752
25%,21.000000,28.000000,20.000000,5.971693
50%,37.000000,51.000000,32.000000,6.425045
75%,84.250000,68.000000,49.000000,6.923643
max,140.000000,145.000000,205.000000,9.935091


In [11]:
soil_df.isnull().sum()

N        0
P        0
K        0
ph       0
label    0
dtype: int64

In [19]:
soil_df["label"].value_counts()

label
rice           100
maize          100
chickpea       100
kidneybeans    100
pigeonpeas     100
mothbeans      100
mungbean       100
blackgram      100
lentil         100
pomegranate    100
banana         100
mango          100
grapes         100
watermelon     100
muskmelon      100
apple          100
orange         100
papaya         100
coconut        100
cotton         100
jute           100
coffee         100
Name: count, dtype: int64

In [18]:
soil_df["label"].value_counts().count()

np.int64(22)

In [4]:
# STEP 2: Load NASA Weather CSVs
# These CSVs were downloaded separately
df_temp = pd.read_csv("T2M.csv", skiprows=9)
df_humid = pd.read_csv("RH2M.csv", skiprows=9)
df_rain = pd.read_csv("PREC.csv", skiprows=9)

# Rename parameter columns for clarity
df_temp.rename(columns={"T2M": "temperature"}, inplace=True)
df_humid.rename(columns={"RH2M": "humidity"}, inplace=True)
df_rain.rename(columns={"PRECTOTCORR": "rainfall"}, inplace=True)

In [6]:
# STEP 3: Merge Weather Data based on LAT, LON, YEAR, DOY
weather = df_temp.merge(df_humid, on=["LAT", "LON", "YEAR", "DOY"])
weather = weather.merge(df_rain, on=["LAT", "LON", "YEAR", "DOY"])

# Optional: Convert DOY (day of year) to full date
weather["date"] = pd.to_datetime(weather["YEAR"].astype(str), format="%Y") + pd.to_timedelta(weather["DOY"] - 1, unit='D')

# Keep only useful columns
weather = weather[["LAT", "LON", "date", "temperature", "humidity", "rainfall"]]

In [8]:
weather.head()

,LAT,LON,date,temperature,humidity,rainfall
0,21.0,85.625,2025-06-01,32.04,62.44,0.40
1,21.0,86.250,2025-06-01,32.01,67.32,0.50
2,21.0,86.875,2025-06-01,31.63,77.71,2.31
3,21.0,87.500,2025-06-01,30.35,87.58,3.40
4,21.0,88.125,2025-06-01,30.37,87.42,4.85


In [12]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   LAT          504 non-null    float64       
 1   LON          504 non-null    float64       
 2   date         504 non-null    datetime64[ns]
 3   temperature  504 non-null    float64       
 4   humidity     504 non-null    float64       
 5   rainfall     504 non-null    float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 23.8 KB


In [13]:
weather.describe()

,LAT,LON,date,temperature,humidity,rainfall
count,504.000000,504.000000,504,504.000000,504.000000,504.000000
mean,23.750000,87.500000,2025-06-03 12:00:00,32.501071,57.303690,2.274901
min,21.000000,85.625000,2025-06-01 00:00:00,27.940000,24.700000,0.000000
25%,22.375000,86.250000,2025-06-02 00:00:00,31.297500,45.132500,0.847500
50%,23.750000,87.500000,2025-06-03 12:00:00,32.285000,57.430000,1.730000
75%,25.125000,88.750000,2025-06-05 00:00:00,33.325000,70.365000,3.227500
max,26.500000,89.375000,2025-06-06 00:00:00,37.630000,90.800000,9.260000
std,1.727741,1.251242,NaN,1.704318,15.874692,1.904811


In [14]:
weather.isnull().sum()

LAT            0
LON            0
date           0
temperature    0
humidity       0
rainfall       0
dtype: int64

In [20]:
# STEP 4: Assign Random Weather Samples to Soil Data
sampled_weather = weather.sample(n=len(soil_df), replace=True, random_state=42).reset_index(drop=True)
combined_df = pd.concat([soil_df.reset_index(drop=True), sampled_weather[["temperature", "humidity", "rainfall"]]], axis=1)

In [22]:
combined_df.head()

,N,P,K,ph,label,temperature,humidity,rainfall
0,90,42,43,6.502985,rice,32.58,69.38,1.49
1,85,58,41,7.038096,rice,31.66,56.90,0.82
2,60,55,44,7.840207,rice,30.53,78.71,3.74
3,74,35,40,6.980401,rice,31.63,74.38,2.34
4,78,42,42,7.628473,rice,32.17,41.30,0.03


In [23]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N            2200 non-null   int64  
 1   P            2200 non-null   int64  
 2   K            2200 non-null   int64  
 3   ph           2200 non-null   float64
 4   label        2200 non-null   object 
 5   temperature  2200 non-null   float64
 6   humidity     2200 non-null   float64
 7   rainfall     2200 non-null   float64
dtypes: float64(4), int64(3), object(1)
memory usage: 137.6+ KB


In [24]:
combined_df.describe()

,N,P,K,ph,temperature,humidity,rainfall
count,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000
mean,50.551818,53.362727,48.149091,6.469480,32.486014,57.263427,2.257136
std,36.917334,32.985883,50.647931,0.773938,1.652885,15.845990,1.890016
min,0.000000,5.000000,5.000000,3.504752,27.940000,24.700000,0.000000
25%,21.000000,28.000000,20.000000,5.971693,31.300000,44.930000,0.870000
50%,37.000000,51.000000,32.000000,6.425045,32.310000,56.800000,1.720000
75%,84.250000,68.000000,49.000000,6.923643,33.340000,69.982500,3.200000
max,140.000000,145.000000,205.000000,9.935091,37.500000,90.800000,9.260000


In [25]:
combined_df.isnull().sum()

N              0
P              0
K              0
ph             0
label          0
temperature    0
humidity       0
rainfall       0
dtype: int64